In [0]:
import os
import dill
import pathlib
import numpy as np
from collections import Counter
import itertools

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
data_path = pathlib.Path("/content/drive/My Drive/validated_clips/balanced_classes/balanced_classes2.pkl")


with open(data_path , "rb") as f:
  data = dill.load(f)

In [0]:
speaker_count = Counter(data['speakers'])

speaker_info= {}

for i in range(len(data['speakers'])):
  speaker = data['speakers'][i]
  accent = data['labels'][i]
  gender = data['genders'][i]

  if speaker not in speaker_info:
    speaker_info[speaker] = {"accent": accent, "gender": gender, "first_ocurrence": i, "num_occurences":1}
  else:
    speaker_info[speaker]["num_occurences"]+=1

In [22]:
print(speaker_info)

{'1514d25318055208d45730bfb6f0391af99036e7778e7880820cc4b9376b707390ceebdfa47ec963fe4345d50eeb76bea2c132f30cd30efbff55487e67ff3e62': {'accent': 'us', 'gender': 'female', 'first_ocurrence': 0, 'num_occurences': 1}, '2ff5127cfa30a1007f613c512819497823d2fd9cd2ff307989ddaeab1aefb6c81845aa2b196b8a70f266993941723143a1d996cb20349b0b82af445436b913a1': {'accent': 'us', 'gender': 'male', 'first_ocurrence': 1, 'num_occurences': 1}, '04960d53cc851eeb6d93f21a09e09ab36fe16943acb226ced1211d7250ab2f1b9a1d655c1cc03d50006e396010851ad52d4c53f49dd77b080b01c4230704c68d': {'accent': 'us', 'gender': 'male', 'first_ocurrence': 2, 'num_occurences': 1}, '0e7bca7f3243636599bd8e7bbe03b4f09ae8898bb0e16e0edc50a2499740c6616dfef740699004ba6bacecad2e3f7f0b52aef60f1dcd01afd81726df6093ad51': {'accent': 'indian', 'gender': 'male', 'first_ocurrence': 3, 'num_occurences': 9}, 'ba15cc6298a85b84cf7a02581b3ff0ea9f525eb69a199dfde74d4277533e53eb7f625b79b50dbfd4ab3e1fb26ffcc0448adf591c09dd115ab91e730a6b4f8a2f': {'accent': 'us', 

In [24]:
counter={"england":{"female":{"speaker_count":0, "audio_count":0}, "male":{"speaker_count":0, "audio_count":0}}, 
         "indian":{"female":{"speaker_count":0, "audio_count":0}, "male":{"speaker_count":0, "audio_count":0}}, 
         "us":{"female":{"speaker_count":0, "audio_count":0}, "male":{"speaker_count":0, "audio_count":0}},  
         "australia":{"female":{"speaker_count":0, "audio_count":0}, "male":{"speaker_count":0, "audio_count":0}}
         }

for info in speaker_info.values():
  accent = info['accent']
  gender = info['gender']
  audio_count = info["num_occurences"]
  counter[accent][gender]['speaker_count']+=1
  counter[accent][gender]['audio_count']+=audio_count

print(counter)

{'england': {'female': {'speaker_count': 39, 'audio_count': 172}, 'male': {'speaker_count': 82, 'audio_count': 164}}, 'indian': {'female': {'speaker_count': 33, 'audio_count': 159}, 'male': {'speaker_count': 110, 'audio_count': 162}}, 'us': {'female': {'speaker_count': 67, 'audio_count': 156}, 'male': {'speaker_count': 62, 'audio_count': 165}}, 'australia': {'female': {'speaker_count': 31, 'audio_count': 169}, 'male': {'speaker_count': 51, 'audio_count': 165}}}


In [0]:
NUM_WANTED_TEST_RECORDINGS_PER_CLASS = 17
NUM_WANTED_VALIDATION_RECORDINGS_PER_CLASS = 17

test_recording_dict = {"england":{"female":{"speaker_list":[], "audio_count":0}, "male":{"speaker_list":[], "audio_count":0}}, 
         "indian":{"female":{"speaker_list":[], "audio_count":0}, "male":{"speaker_list":[], "audio_count":0}}, 
         "us":{"female":{"speaker_list":[], "audio_count":0}, "male":{"speaker_list":[], "audio_count":0}},  
         "australia":{"female":{"speaker_list":[], "audio_count":0}, "male":{"speaker_list":[], "audio_count":0}}
         }
validation_recording_dict = {"england":{"female":{"speaker_list":[], "audio_count":0}, "male":{"speaker_list":[], "audio_count":0}}, 
         "indian":{"female":{"speaker_list":[], "audio_count":0}, "male":{"speaker_list":[], "audio_count":0}}, 
         "us":{"female":{"speaker_list":[], "audio_count":0}, "male":{"speaker_list":[], "audio_count":0}},  
         "australia":{"female":{"speaker_list":[], "audio_count":0}, "male":{"speaker_list":[], "audio_count":0}}
         }


for key,value in speaker_info.items():
  accent = value['accent']
  gender = value['gender']
  audio_count = value["num_occurences"]
  if test_recording_dict[accent][gender]['audio_count'] + audio_count <= NUM_WANTED_TEST_RECORDINGS_PER_CLASS:
    test_recording_dict[accent][gender]["speaker_list"].append(key)
    test_recording_dict[accent][gender]['audio_count'] += audio_count
  elif validation_recording_dict[accent][gender]['audio_count'] + audio_count <= NUM_WANTED_TEST_RECORDINGS_PER_CLASS:
    validation_recording_dict[accent][gender]["speaker_list"].append(key)
    validation_recording_dict[accent][gender]['audio_count'] += audio_count

In [0]:
accents = ["england", "indian", "us", "australia"]

test_speakers = []
for accent in accents:
  for gender in ["male", "female"]:
    test_speakers.extend(test_recording_dict[accent][gender]["speaker_list"])

val_speakers = []
for accent in accents:
  for gender in ["male", "female"]:
    val_speakers.extend(validation_recording_dict[accent][gender]["speaker_list"])

In [43]:
train_data ={
    'speakers':[],
    'labels':[],
    'genders':[],
    'mfccs':[],
    'paths':[]
    }
test_data ={
    'speakers':[],
    'labels':[],
    'genders':[],
    'mfccs':[],
    'paths':[]
    }
val_data ={
    'speakers':[],
    'labels':[],
    'genders':[],
    'mfccs':[],
    'paths':[]
    }

for i in range(len(data["speakers"])):
  speaker = data['speakers'][i]
  accent = data['labels'][i]
  gender = data['genders'][i]
  mfcc = data['mfccs'][i]
  path = data['paths'][i]

  if speaker in test_speakers:
    test_data['speakers'].append(speaker)
    test_data['labels'].append(accent)
    test_data['genders'].append(gender)
    test_data['mfccs'].append(mfcc)
    test_data['paths'].append(path)

  elif speaker in val_speakers:
    val_data['speakers'].append(speaker)
    val_data['labels'].append(accent)
    val_data['genders'].append(gender)
    val_data['mfccs'].append(mfcc)
    val_data['paths'].append(path)

  else:
    train_data['speakers'].append(speaker)
    train_data['labels'].append(accent)
    train_data['genders'].append(gender)
    train_data['mfccs'].append(mfcc)
    train_data['paths'].append(path)

print([len(value) for value in test_data.values()])
print([len(value) for value in val_data.values()])
print([len(value) for value in train_data.values()])

[136, 136, 136, 136, 136]
[136, 136, 136, 136, 136]
[1040, 1040, 1040, 1040, 1040]


In [0]:
balanced_classes_folder = pathlib.Path("/content/drive/My Drive/validated_clips/balanced_classes/")

test_path = balanced_classes_folder / "test_data.pkl"
val_path = balanced_classes_folder / "val_data.pkl"
train_path = balanced_classes_folder / "train_data.pkl"

with open(test_path , "wb") as f:
  dill.dump(test_data, f)

with open(val_path , "wb") as f:
  dill.dump(val_data, f)

with open(train_path , "wb") as f:
  dill.dump(train_data, f)